In [1]:
import os
from datetime import datetime
from pathlib import Path

from core.runner import Runner
from core.environment import SingleVertexSpinFoam, StarModelSpinFoam


def train_gfn(
    spin_j,
    sf_model,
    main_layer_hidden_nodes,
    branch1_hidden_nodes,
    branch2_hidden_nodes,
    activation,
    exploration_rate,
    training_fraction_from_back_traj,
    learning_rate,
    batch_size,
    n_iterations,
    evaluation_batch_size,
    generate_samples_every_m_training_samples,
):
    if sf_model == "single_vertex_model":
        spinfoam_model = SingleVertexSpinFoam(spin_j=spin_j)
    elif sf_model == "star_model":
        spinfoam_model = StarModelSpinFoam(spin_j=spin_j)
    else:
        raise ValueError(
            "Spinfoam model not yet implemented. " "Custom Spinfoam class can be made."
        )

    # training_run_datetime = datetime.now().strftime("%B %d, %Y at %H:%M:%S")

    directory_for_generated_samples = Path(
        f"./data/",
        f"GFlowNet/{sf_model}/j_{spin_j}/n_iterations_{n_iterations}/",
        f"main_layer_hid_nodes_{main_layer_hidden_nodes}/exploration_rate_{exploration_rate}/learning_rate_{learning_rate}/",
        f"batch_size_{batch_size}/",
        f"samples"
    )

    runner = Runner(
        spinfoam_model=spinfoam_model,
        main_layer_hidden_nodes=main_layer_hidden_nodes,
        branch1_hidden_nodes=branch1_hidden_nodes,
        branch2_hidden_nodes=branch2_hidden_nodes,
        activation=activation,
        exploration_rate=exploration_rate,
        training_fraction_from_back_traj=training_fraction_from_back_traj,
        learning_rate=learning_rate,
    )

    ave_losses = runner.train_agent(
        batch_size,
        n_iterations,
        evaluation_batch_size,
        generate_samples_every_m_training_samples,
        directory_for_generated_samples,
    )

    return ave_losses

2023-05-11 12:36:22.166704: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-11 12:36:22.689583: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-11 12:36:23.670764: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/frisus/Scrivania/sl2cfoam-next-dev/lib:/home/frisus/anaconda3/envs/tf/lib/
2023-05-11 12:36:23.670954: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvi

## Vertex Amplitude

In [2]:
spin = 2.0
model = "single_vertex_model"
main_layer_hidden_nodes = (30, 20)
branch1_hidden_nodes = ()
branch2_hidden_nodes = ()
activation = "swish"
exploration_rate = 0.5
training_fraction_from_back_traj = 0.0
learning_rate = 0.0005
batch_size = int(1e3)
n_iterations = int(1e4)
evaluation_batch_size = int(1e6)
generate_samples_every_m_training_samples = int(1e6)

In [3]:
ave_losses = train_gfn(
    spin,
    model,
    main_layer_hidden_nodes,
    branch1_hidden_nodes,
    branch2_hidden_nodes,
    activation,
    exploration_rate,
    training_fraction_from_back_traj,
    learning_rate,
    batch_size,
    n_iterations,
    evaluation_batch_size,
    generate_samples_every_m_training_samples,
)

2023-05-11 12:37:35.768889: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-11 12:37:35.805079: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-11 12:37:35.805456: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-11 12:37:35.808129: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Nth iteration: 1000 Trained on K samples: 1000000 Average Loss: 2.243118234368096
Nth iteration: 2000 Trained on K samples: 2000000 Average Loss: 1.9743874586520318
Nth iteration: 3000 Trained on K samples: 3000000 Average Loss: 1.3092901882694201
Nth iteration: 4000 Trained on K samples: 4000000 Average Loss: 1.0606379924388143
Nth iteration: 5000 Trained on K samples: 5000000 Average Loss: 0.66560984732873729
Nth iteration: 6000 Trained on K samples: 6000000 Average Loss: 0.683636163682316
Nth iteration: 7000 Trained on K samples: 7000000 Average Loss: 0.46280469184088496
Nth iteration: 8000 Trained on K samples: 8000000 Average Loss: 0.44634887096808651
Nth iteration: 9000 Trained on K samples: 9000000 Average Loss: 0.47803748546741909
Nth iteration: 10000 Trained on K samples: 10000000 Average Loss: 0.41243994232075115
INFO:tensorflow:Assets written to: /home/frisus/Scrivania/GFlowNets_on_SpinFoams/data/GFlowNet/single_vertex_model/j_2.0/n_iterations_10000/main_layer_hid_nodes_(30,